# File ingestion and schema validation
Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dask.dataframe as dd


reading the data in pandas

In [4]:
start_time = time.process_time()
df_pd=pd.read_csv(r'2019.csv', encoding='latin-1')
print(time.process_time() - start_time, "seconds")

79.609375 seconds


In [6]:
df_pd.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


reading with Dask

In [13]:
start_time = time.process_time()
df_dask=dd.read_csv('2019.csv',encoding='latin-1')
#df_dask = df_dask.repartition()
#df_dask_computed=df_dask.compute()
print(time.process_time() - start_time, "seconds")

0.015625 seconds


In [14]:
df_dask.shape, df_dask


((Delayed('int-2caf9f34-2b1e-4148-a9ac-0c3108f719e5'), 9),
 Dask DataFrame Structure:
                event_time event_type product_id category_id category_code   brand    price user_id user_session
 npartitions=89                                                                                                 
                    object     object      int64       int64        object  object  float64   int64       object
                       ...        ...        ...         ...           ...     ...      ...     ...          ...
 ...                   ...        ...        ...         ...           ...     ...      ...     ...          ...
                       ...        ...        ...         ...           ...     ...      ...     ...          ...
                       ...        ...        ...         ...           ...     ...      ...     ...          ...
 Dask Name: read-csv, 89 tasks)

Reading Modin with Dask

In [ ]:
import os
os.environ["MODIN_ENGINE"] = "dask"
import modin.pandas as pd

In [17]:

from dask.distributed import Client
client = Client()
start_time = time.process_time()
df_modin_dask=pd.read_csv('2019.csv',encoding='latin-1')
print(time.process_time() - start_time, "seconds")

24.046875 seconds


Reading Modin with Ray

In [20]:

os.environ["MODIN_ENGINE"] = "ray"
start_time = time.process_time()
df_modin_ray=pd.read_csv('2019.csv',encoding='latin-1')
print(time.process_time() - start_time, "seconds")

distributed.utils_perf - WARNING - full garbage collections took 98% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 98% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 98% CPU time recently (threshold: 10%)


25.84375 seconds


From the above code, we found that dask was the fastest method for reading the file.

 check how many missing values there are for each column

In [29]:
df_pd.isna().sum()

distributed.utils_perf - WARNING - full garbage collections took 95% CPU time recently (threshold: 10%)


event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

distributed.utils_perf - WARNING - full garbage collections took 95% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 93% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 93% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 93% CPU time recently (threshold: 10%)


Removed columns with very high number of missing values

In [12]:
df_main=df_pd



In [13]:
del df_main['category_code']
del df_main['brand']

In [14]:
df_main.columns = df_main.columns.str.replace(' ', '')


In [15]:
df_main=df_main.dropna()

In [53]:
df_main.head()


,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
df_main.to_csv('df_main.csv', index=False)

distributed.utils_perf - WARNING - full garbage collections took 97% CPU time recently (threshold: 10%)


Orginal data 

In [12]:
from pathlib import Path
Path('2019.csv').stat().st_size

5668612855

In [4]:
df_pd.shape


(42448764, 9)

In [5]:
df_pd.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

Cleaned data


In [13]:
Path('df_main.csv').stat().st_size

5127323759

In [11]:
df_main.shape

(42448762, 7)

In [55]:
df_main.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'price',
       'user_id', 'user_session'],
      dtype='object')

In [18]:
df_main

,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59 UTC,view,21407424,2053013561579406073,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


YAML FILE CREATION

In [23]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [24]:
%%writefile file.yaml
file_type: csv
dataset_name: df_main
file_name: example
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 0
columns: 
    - product_id
    - category_id
    - price
    - user_id
    - user_session
    - event_type
    - event_time


Overwriting file.yaml


In [25]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [26]:
config_data['inbound_delimiter']

','

In [27]:
config_data


{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'example',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 0,
 'columns': ['product_id',
  'category_id',
  'price',
  'user_id',
  'user_session',
  'event_type',
  'event_time']}

In [28]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

C:\Users\cp496\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [29]:

# validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [30]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['event_time', 'event_type', 'product_id', 'category_id', 'price',
       'user_id', 'user_session'],
      dtype='object')
columns of YAML are: ['product_id', 'category_id', 'price', 'user_id', 'user_session', 'event_type', 'event_time']


In [32]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [33]:
util.col_header_val(df_main,config_data)


column name and column length validation passed


1

Write the file in pipe separated text file (|) in gz format



In [35]:
df.to_csv('df_gz.csv.gz', compression='gzip')


In [39]:
dgz = pd.read_csv("df_gz.csv.gz")


In [40]:
dgz.head()

,Unnamed: 0,event_time,event_type,product_id,category_id,price,user_id,user_session
0,0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [41]:
dgz.columns


Index(['Unnamed: 0', 'event_time', 'event_type', 'product_id', 'category_id',
       'price', 'user_id', 'user_session'],
      dtype='object')

In [42]:
dgz.shape


(42448762, 8)

In [44]:
from pathlib import Path
Path('df_main.csv').stat().st_size

4756396009

In [45]:
from pathlib import Path
Path('df_gz.csv.gz').stat().st_size

1644731641

The compressed version's size is much smaller than the original one.